<a href="https://colab.research.google.com/github/citruscaesar/building-footprint-extraction/blob/main/experiments.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Download Dependencies

In [ ]:
%%capture
!pip install wandb
!pip install lightning
!pip install torchmetrics
!pip install segmentation-models-pytorch
!pip install rasterio

In [ ]:
!git clone https://github.com/citruscaesar/building-footprint-extraction.git
!mkdir -p /content/src
!cp -r /content/building-footprint-extraction/src/ /content/

Cloning into 'building-footprint-extraction'...
remote: Enumerating objects: 25, done.
remote: Counting objects: 100% (25/25), done.
remote: Compressing objects: 100% (24/24), done.
remote: Total 25 (delta 5), reused 13 (delta 0), pack-reused 0
Unpacking objects: 100% (25/25), 1.77 MiB | 10.25 MiB/s, done.


In [ ]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!mkdir -p ./inria/train

In [ ]:
!ls

building-footprint-extraction  drive  inria  sample_data  src


In [ ]:
%%capture
'''
#!cp -r /content/drive/MyDrive/datasets/inria/AerialImageDataset/train /content/inria/train
#!cp -r /content/drive/MyDrive/datasets/inria/AerialImageDataset/tiled/* /content/inria/tiled
#!mkdir -p inria/tiled
'''

!mkdir -p ./inria/train
!cp /content/drive/MyDrive/datasets/inria/archive/tiled.zip ./inria  
!unzip inria/tiled.zip -d ./inria/

In [ ]:
!ls ./inria/

tiled  tiled.zip  train


In [ ]:
from pathlib import Path
tiled = {
    "drive": Path("/content/drive/MyDrive/datasets/inria/AerialImageDataset/tiled/512x512"),
    "runtime": Path("/content/inria/tiled/512x512")
}
train_scenes = tiled["runtime"] / "train" / "scenes"

In [ ]:
assert (len(list(train_scenes.iterdir()))) == 18600

### Imports and Config

In [ ]:
import os
from pathlib import Path
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import rasterio as rio

import torch
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms

import pytorch_lightning as pl
import torchmetrics 
from torchmetrics import MetricCollection
from torchmetrics.classification import BinaryJaccardIndex, BinaryConfusionMatrix, BinaryF1Score, BinaryCohenKappa

import wandb
from pytorch_lightning.loggers import WandbLogger, CSVLogger

import segmentation_models_pytorch as smp
from segmentation_models_pytorch.encoders import get_preprocessing_fn

from src.datamodule import InariaDataModule

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Pytorch Version: {torch.__version__}")
print(f"Lightning Version: {pl.__version__}")
print(f"Pytorch Running on :{DEVICE}")
#print(torch.__config__.show())

Pytorch Version: 2.0.0+cu118
Lightning Version: 2.0.1.post0
Pytorch Running on :cuda


In [ ]:
DATA = Path.cwd() / "inria" / "train" 
#DATA = Path("/content/drive/MyDrive/datasets/inria/AerialImageDataset/train")
#DATA = Path("/media/sambhav/30AC4696AC46568E/datasets/inaria/train")

In [ ]:
list(DATA.iterdir())
list((DATA.parent / "tiled").iterdir())

[PosixPath('/content/inria/tiled/512x512')]

### Segmentation Module

In [ ]:
class SegModule(pl.LightningModule):
    def __init__(self, model, preprocess, learning_rate, batch_size):
        super().__init__()
        self.model = model
        self.preprocess_fn = preprocess
        self.loss = smp.losses.DiceLoss(smp.losses.BINARY_MODE, from_logits = True)

        self.save_hyperparameters(ignore = ["model", "preprocess"])

        self.val_metrics = MetricCollection([
            BinaryF1Score(),
            BinaryJaccardIndex(),
            BinaryCohenKappa(),
        ], prefix = "Validation ")

        self.test_metrics = MetricCollection([
            BinaryF1Score(),
            BinaryJaccardIndex(),
            BinaryCohenKappa(),
        ], prefix = "Test ")


    def forward(self, batch):
        mask = self.model(batch) 
        return mask
        
    def training_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask) 
        self.log("train/loss", loss.item(), on_epoch = True, on_step=True, prog_bar=True)
        return loss
    
    def validation_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask)
        self.log("val/loss", loss.item(), on_epoch=True)

        self.val_metrics.update(pred_mask, mask)
        self.log_dict(self.val_metrics, on_epoch=True)

    def test_step(self, batch, batch_idx):
        scene, mask = batch
        pred_mask = self.model(scene)
        loss = self.loss(pred_mask, mask)
        self.log("test/loss", loss.item(), on_epoch=True)

        self.test_metrics.update(pred_mask, mask)
        self.log_dict(self.test_metrics, on_epoch=True)

    def configure_optimizers(self):
        optimizer = torch.optim.Adam(self.model.parameters(), lr = self.hparams.learning_rate)
        return optimizer

### Experiments

In [ ]:
#TODO: Create Config Object, Setup to Return Config Dict 
#config dict
host = "colab"
assert host in ["local", "colab"]
config = {
    "colab": {
        "tile_shape": (512, 512),
        "batch_size": 32,
        "accumulate": 0,
        "num_workers": 2,
        "learning_rate": 3e-4,
        "num_epochs": 5,
        "seed": 69
    },

    "local": {
        "batch_size": 8,
        "accumulate": 1,
        "num_workers": 16,
        "learning_rate": 3e-4,
        "num_epochs": 1
    }
}
config = config[host]

In [ ]:
pl.seed_everything(seed = config["seed"], workers = True)

INFO:lightning_fabric.utilities.seed:Global seed set to 69


69

In [ ]:
#os.environ["WANDB_NOTEBOOK_NAME"] = "experiments"
wandb.finish()
logs_dir = Path.cwd() / "logs"
logs_dir.mkdir(exist_ok = True)
wandblogger = WandbLogger(
    project = "building-segmentation-tests",
    save_dir = logs_dir 
) 

<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize
wandb: Paste an API key from your profile and hit enter, or press ctrl+c to quit:

 ··········


wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc


In [ ]:
unet = smp.Unet("resnet18", encoder_weights="imagenet", in_channels=3, classes = 1)
preprocess = get_preprocessing_fn("resnet18", "imagenet")

In [ ]:
model = SegModule(unet, preprocess, learning_rate = 3e-4, batch_size = config["batch_size"])

In [ ]:
dm = InariaDataModule(DATA, tile_shape = config["tile_shape"], batch_size = config["batch_size"], num_workers = config["num_workers"])

In [ ]:
%%capture
dev_trainer = pl.Trainer(
    accelerator = "auto",
    fast_dev_run = 1
)

trainer = pl.Trainer(
    accelerator = "auto",
    #accumulate_grad_batches = config["accumulate"],
    #limit_train_batches = .25,
    max_epochs = config["num_epochs"],
    precision = 16,
    logger = [wandblogger],
)

INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs
INFO:pytorch_lightning.utilities.rank_zero:Running in `fast_dev_run` mode: will run the requested loop using 1 batch(es). Logging and checkpointing is suppressed.
INFO:pytorch_lightning.utilities.rank_zero:Using 16bit Automatic Mixed Precision (AMP)
INFO:pytorch_lightning.utilities.rank_zero:GPU available: True (cuda), used: True
INFO:pytorch_lightning.utilities.rank_zero:TPU available: False, using: 0 TPU cores
INFO:pytorch_lightning.utilities.rank_zero:IPU available: False, using: 0 IPUs
INFO:pytorch_lightning.utilities.rank_zero:HPU available: False, using: 0 HPUs


In [ ]:
dev_trainer.fit(model, datamodule = dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params
--------------------------------------------------
0 | model        | Unet             | 14.3 M
1 | loss         | DiceLoss         | 0     
2 | val_metrics  | MetricCollection | 0     
3 | test_metrics | MetricCollection | 0     
--------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.313    Total estimated model params size (MB)


Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

INFO:pytorch_lightning.utilities.rank_zero:`Trainer.fit` stopped: `max_steps=1` reached.


In [ ]:
trainer.fit(model, datamodule = dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]
INFO:pytorch_lightning.callbacks.model_summary:
  | Name         | Type             | Params
--------------------------------------------------
0 | model        | Unet             | 14.3 M
1 | loss         | DiceLoss         | 0     
2 | val_metrics  | MetricCollection | 0     
3 | test_metrics | MetricCollection | 0     
--------------------------------------------------
14.3 M    Trainable params
0         Non-trainable params
14.3 M    Total params
57.313    Total estimated model params size (MB)


Sanity Checking: 0it [00:00, ?it/s]

Training: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

Validation: 0it [00:00, ?it/s]

/usr/local/lib/python3.9/dist-packages/pytorch_lightning/trainer/call.py:54: UserWarning: Detected KeyboardInterrupt, attempting graceful shutdown...
  rank_zero_warn("Detected KeyboardInterrupt, attempting graceful shutdown...")


In [ ]:
trainer.test(model, datamodule = dm)

INFO:pytorch_lightning.accelerators.cuda:LOCAL_RANK: 0 - CUDA_VISIBLE_DEVICES: [0]


Testing: 0it [00:00, ?it/s]

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃        Test metric        ┃       DataLoader 0        ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│   Test BinaryCohenKappa   │    0.8377863168716431     │
│    Test BinaryF1Score     │     0.860415518283844     │
│  Test BinaryJaccardIndex  │    0.7550256848335266     │
│         test/loss         │    0.14075955748558044    │
└───────────────────────────┴───────────────────────────┘

[{'test/loss': 0.14075955748558044,
  'Test BinaryF1Score': 0.860415518283844,
  'Test BinaryJaccardIndex': 0.7550256848335266,
  'Test BinaryCohenKappa': 0.8377863168716431}]

In [ ]:
trainer.save_checkpoint("./segmodel-checkpoint.ckpt")

In [ ]:
!cp ./segmodel-checkpoint.ckpt /content/drive/MyDrive/datasets/inria/

In [ ]:
wandb.finish()

Test BinaryCohenKappa,▁
Test BinaryF1Score,▁
Test BinaryJaccardIndex,▁
Validation BinaryCohenKappa,▁▄▅▆▇█
Validation BinaryF1Score,▁▄▅▆▇█
Validation BinaryJaccardIndex,▁▄▅▆▇█
epoch,▁▁▁▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▅▅▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇██
test/loss,▁
train/loss_epoch,█▃▂▂▁▁
train/loss_step,█▅▅▄▄▃▂▂▃▂▂▂▂▂▂▂▂▂▂▁▁▂▁▂▂▂▃▂▂▃▂▂▂▂▃▂▂▁▂▂
trainer/global_step,▁▁▁▂▂▂▂▂▂▃▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▇▇▇▇▇▇████


In [ ]:
drive.flush_and_unmount()